In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
from pathlib import Path

import pdblp
import numpy as np
import pandas as pd

if (path := 'C:\\dev\\projects\\credit\\backend') not in sys.path:
    sys.path.append(path)

from api.blp import BlpQuery
from api.data.fields import fields_reference
from api.data.processing import post_process
from api.data.base import tables
import utils

In [ ]:
timestamp = f"{pd.Timestamp.today():%Y%m%d_%H%M%S}"

In [ ]:
root_path = Path('/dev/projects/credit/backend/')
data_path = root_path / 'data'

### Bloomberg Connection

In [ ]:
bq = BlpQuery().start()

In [ ]:
bond_ref = tables.bonds_reference

In [ ]:
cds_5y = (bond_ref['cds_ticker_5y']).dropna().drop_duplicates().to_list()

In [ ]:
field = 'SECURITY_DES'
res = bq.bdp(
    securities=cds_5y,
    fields=[field]
)

In [ ]:
from itertools import product
years = [2, 5, 7, 10]
capital_stack = ['SR', 'SUB', 'SLA']

def get_cds_generic_name(security):
    field = 'CDS_SPREAD_TICKER_5Y'
    cds_generic_ticker = bq.bdp(
        securities=[security],    
        fields=[field]
    )
    cds_generic_ticker = cds_generic_ticker[field].squeeze() + ' Curncy'

    field = 'SECURITY_DES'
    cds_generic_name = bq.bdp(
        securities=[cds_generic_ticker],    
        fields=[field]
    )
    cds_generic_name = cds_generic_name[field].squeeze()
    return cds_generic_name

def get_cds_tickers(
    security: str = None,
    cds_generic_name=None,
    bq=None
):
    
    if security and not cds_generic_name:
        cds_generic_name = get_cds_generic_name(security)
    cds_generic_name = ' '.join(cds_generic_name.split(' ', 3)[:-1])
    tickers = []
    for yrs, cs in product(years, capital_stack):
        cds_ticker = f"{cds_generic_name} {cs} {yrs}Y D14 Curncy"
        tickers.append({'name': cds_ticker , 'years': yrs, 'capital_stack': cs})
    return pd.DataFrame(tickers)

In [ ]:
cds_ref = []
for index, row in res.iterrows():
    cds_ticker_5y, cds_generic_name = row['security'], row[field]
    cds_tickers = get_cds_tickers(cds_generic_name=cds_generic_name)
    cds_tickers['cds_ticker_5y'] = cds_ticker_5y
    cds_ref.append(cds_tickers)
cds_ref = pd.concat(cds_ref)

In [ ]:
to_merge = bond_ref[['ticker', 'issuer_equity_ticker', 'cds_ticker_5y']].dropna().drop_duplicates(subset='cds_ticker_5y').set_index('cds_ticker_5y')

In [ ]:
cds_ref = cds_ref.merge(to_merge, how='left', left_on='cds_ticker_5y', right_index=True)

In [ ]:
cds_ref.to_csv(data_path / 'cds_reference.csv', index=False)

In [ ]:
cds_prices = [] 
cds_na = []

for sec in cds_ref['name']:
    try:
        res = bq.bdp(
            securities=[sec],
            fields=['PX_LAST']
        )
        print(f'{sec} ---> OK')
        cds_prices.append(res)
    except TypeError:
        cds_na.append(sec)
        print(f'{sec} ---> NA')

BBVA CDS EUR SR 2Y D14 Curncy ---> OK
BBVA CDS EUR SUB 2Y D14 Curncy ---> OK
BBVA CDS EUR SLA 2Y D14 Curncy ---> OK
BBVA CDS EUR SR 5Y D14 Curncy ---> OK
BBVA CDS EUR SUB 5Y D14 Curncy ---> OK
BBVA CDS EUR SLA 5Y D14 Curncy ---> OK
BBVA CDS EUR SR 7Y D14 Curncy ---> OK
BBVA CDS EUR SUB 7Y D14 Curncy ---> OK
BBVA CDS EUR SLA 7Y D14 Curncy ---> OK
BBVA CDS EUR SR 10Y D14 Curncy ---> OK
BBVA CDS EUR SUB 10Y D14 Curncy ---> OK
BBVA CDS EUR SLA 10Y D14 Curncy ---> OK
DANBNK CDS EUR SR 2Y D14 Curncy ---> OK
DANBNK CDS EUR SUB 2Y D14 Curncy ---> OK
DANBNK CDS EUR SLA 2Y D14 Curncy ---> OK
DANBNK CDS EUR SR 5Y D14 Curncy ---> OK
DANBNK CDS EUR SUB 5Y D14 Curncy ---> OK
DANBNK CDS EUR SLA 5Y D14 Curncy ---> OK
DANBNK CDS EUR SR 7Y D14 Curncy ---> OK
DANBNK CDS EUR SUB 7Y D14 Curncy ---> OK
DANBNK CDS EUR SLA 7Y D14 Curncy ---> OK
DANBNK CDS EUR SR 10Y D14 Curncy ---> OK
DANBNK CDS EUR SUB 10Y D14 Curncy ---> OK
DANBNK CDS EUR SLA 10Y D14 Curncy ---> OK
AIB CDS EUR SR 2Y D14 Curncy ---> OK
AIB C

In [ ]:
cds_prices = pd.concat(cds_prices)

In [ ]:
cds_ref['exists'] = ~cds_ref['name'].isin(cds_na)

In [ ]:
cds_ref.to_csv(data_path / 'cds_reference.csv', index=False)

In [ ]:
cds_prices.to_csv(data_path / 'cds_market.csv', index=False)